<a href="https://colab.research.google.com/github/cookieDroid/jollyRancher/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader

###Dependency
Install the python dependencies

In [ ]:
#@title
!apt install python-libtorrent
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
import ipywidgets as widgets
from google.colab import drive
import libtorrent as lt
import time
import datetime
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, ToggleButton
from ipywidgets import RadioButtons as rd
from IPython.display import display

### Mount Google Drive
To stream files we need to mount Google Drive and after downloading select unmount google drive

In [ ]:
Mode = rd(options=['MOUNT', 'UNMOUNT'],value='MOUNT', description='Drive Mount :',disabled=False)
display(Mode)

In [ ]:
MODE = Mode.value
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
  print("Mounted")
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
    print("Unmounted")
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

### Set the location to download. If you have shared drive select it and enter the name

In [ ]:
Location = rd(options=['My Drive', 'Shared Drive'], description='Drive type :',disabled=False)
display(Location)

In [ ]:
location = Location.value
print(location)
if location == "Shared Drive":
  sharedDriveName = input("Enter Shared Drive Name : ")
  filepath='/content/drive/Shared drives/'+sharedDriveName+'/Torrent/'
else:
  filepath='/content/drive/My Drive/Torrent/'
print(filepath)

### Initiate torrent client app


In [ ]:
#@title
ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': filepath,
    'storage_mode': lt.storage_mode_t(2),
    }

### Enter the magnet/torrent link or upload torrent file

In [ ]:
linkType = rd(options=['Link', 'Torrent File'],value='Link', description='Torrent type :',disabled=False)
display(linkType)

In [ ]:
if(linkType.value=="Link"):
  link=input("Enter the Link")
else:
  link=widgets.FileUpload(accept='.torrent', multiple=False)
  display(link)


### Process the torrent file and download metadata

In [ ]:

handle = lt.add_magnet_uri(ses, link, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
ses.pause()

#printing file name
info = handle.get_torrent_info()
togglebox = [''] * info.files().num_files()
optionbox = [''] * info.files().num_files()


### Select the files to download

In [ ]:
print("Select the files to download")
for x in range(info.files().num_files()):
    togglebox[x]=info.files().file_path(x)
    optionbox[x]= widgets.Checkbox(value=False,disabled=False,indent=False) 
    display(optionbox[x],togglebox[x])

### Run the program

In [ ]:
for x in range(info.files().num_files()):
  if optionbox[x].value==True:
    handle.file_priority(x,1)
  else:
    handle.file_priority(x,0)
ses.resume()
print("Started Download...")
while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating']
    print ('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]))
    if s.progress>=1:
        
        break;

    time.sleep(5)

end = time.time()
print("COMPLETE","/n","check google drive for if files are downloaded or not")
print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")
print(datetime.datetime.now())